# Eigenvectors and Eigenvalues 


### Jacobi transformations

In [1]:
import torch
from torch.utils.cpp_extension import load

In [5]:
jacobi = load(name='jacobi',
             build_directory='./build',
             sources=['jacobi.cc'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             verbose=False)

Jacobi rotation for rows/columns $p$ and $q$:
$$
  P_{pq}(\phi) = \begin{pmatrix}
   1 &        &           &        &          &        &   \\
     & \cdots &           &        &          &        &   \\
     &        & \cos\phi  & \cdots & \sin\phi &        &   \\
     &        &  \vdots   &  1     &  \vdots  &        &   \\
     &        & -\sin\phi & \cdots & \cos\phi &        &   \\
     &        &           &        &          & \cdots &   \\
     &        &           &        &          &        & 1 \\
  \end{pmatrix}
$$

Let $A$ a real symmetric matrix, apply the Jacobi transformation:
$$
A' = P^T_{pq}(\phi) \cdot A \cdot P_{pq}(\phi)
$$


### Exercises
**(1)** Show that:
$$
a'_{pq} = (\cos^2\phi - \sin^2\phi)a_{pq} + (a_{pp} - a_{qq})\cos\phi\sin\phi
$$

**(2)** Requiring $a'_{pq}=0$ amounts to solve:
$$
\tan^2\phi + 2\theta\tan\phi-1 = 0
$$
where 
$$
\theta = \frac{a_{qq} - a_{pp}}{2 a_{pq}}
$$

**(3)** A solution $\phi_0 < \pi / 4$ is given by:
$$
\tan\phi_0 = \frac{\mathrm{sgn}(\theta)}{ \left|\theta\right| + \sqrt{\theta^2+1} }
$$

**(4)** Defining:
$$
S(A) = \sum_{r \neq s} \left|a_{rs}\right|^2, 
$$
show that for $\phi_0$:
$$
S(A') = S(A) - 2\left|a_{pq}\right|^2
$$

Applying iteratively Jacobi transformations, the above equation guarantees convergence to orthonormal eigenvectors:
$$
V = P_1 \cdot P_2 \cdots I
$$
with corresponding eigenvalues on the diagonal:
$$
D = V^T \cdot A \cdot V
$$



In [6]:
A_ = torch.randn((5,5)).double()
A = A_ + A_.t()
A

tensor([[ 0.0422, -1.3537,  0.0888,  1.0250, -1.3220],
        [-1.3537, -3.1583,  0.2201,  0.7171, -0.3102],
        [ 0.0888,  0.2201,  3.8792,  0.6300,  1.8657],
        [ 1.0250,  0.7171,  0.6300,  0.8658, -0.2857],
        [-1.3220, -0.3102,  1.8657, -0.2857,  0.3378]], dtype=torch.float64)

In [8]:
D, V = jacobi.symeig(A)

In [9]:
D

tensor([-1.0043, -4.0293,  4.7675,  2.1004,  0.1323], dtype=torch.float64)

In [10]:
V

tensor([[ 0.4106,  0.4032, -0.0815,  0.6162, -0.5315],
        [-0.3563,  0.8706,  0.0328, -0.0385,  0.3355],
        [-0.3031, -0.0598,  0.9076,  0.1563, -0.2374],
        [ 0.1316, -0.1928,  0.1021,  0.6609,  0.7060],
        [ 0.7715,  0.1968,  0.3977, -0.3970,  0.2241]], dtype=torch.float64)

In [11]:
Ds, Di = D.sort()
Ds

tensor([-4.0293, -1.0043,  0.1323,  2.1004,  4.7675], dtype=torch.float64)

In [12]:
V.t()[Di].t()

tensor([[ 0.4032,  0.4106, -0.5315,  0.6162, -0.0815],
        [ 0.8706, -0.3563,  0.3355, -0.0385,  0.0328],
        [-0.0598, -0.3031, -0.2374,  0.1563,  0.9076],
        [-0.1928,  0.1316,  0.7060,  0.6609,  0.1021],
        [ 0.1968,  0.7715,  0.2241, -0.3970,  0.3977]], dtype=torch.float64)

In [13]:
e, vs = torch.linalg.eigh(A)
e

tensor([-4.0293, -1.0043,  0.1323,  2.1004,  4.7675], dtype=torch.float64)

In [14]:
vs

tensor([[ 0.4032,  0.4106, -0.5315,  0.6162, -0.0815],
        [ 0.8706, -0.3563,  0.3355, -0.0385,  0.0328],
        [-0.0598, -0.3031, -0.2374,  0.1563,  0.9076],
        [-0.1928,  0.1316,  0.7060,  0.6609,  0.1021],
        [ 0.1968,  0.7715,  0.2241, -0.3970,  0.3977]], dtype=torch.float64)

### Exercises 
**(1)** Let $A$ and $B$ be a real symmetric matrices, with $B$ positive definite. Implement an efficient routine to solve the generalised eigenvalue problem:
$$
A \cdot x = \lambda B \cdot x
$$

**(2)** Implement a routine to that attempts to solve the non-linear eigenvalue problem:
$$
(\lambda^2 A + \lambda B + C) \cdot x = 0
$$
over the reals, for any compatible set of square matrices.
